In [9]:
import pandas as pd


In [1]:
from dotenv import load_dotenv
import os

load_dotenv()  # load variables from .env

mysql_user = os.getenv("MYSQL_USER")
mysql_pass = os.getenv("MYSQL_PASSWORD")
mysql_host = os.getenv("MYSQL_HOST")
mysql_db = os.getenv("MYSQL_DB")
api_key = os.getenv("API_KEY")


In [10]:
sales = pd.DataFrame({
    "order_id" : [1,2,3,4],
    "customer_id" : [1,2,3,1],
    "product_id" : [101, 102, 103, 104],
    "amount" : [200, 150, 300, 250]
})


sales.to_csv("../data/sales.csv", index=False)

In [11]:
customers = pd.DataFrame({
    "customer_id" : [1,2,3],
    "name" : ["Alice", "Bob", "Charlie"],
    "region" : ["KL", "Penang", "Johor"]
})

customers.to_csv("../data/customers.csv", index=False)

In [12]:
returns = pd.DataFrame({
    "order_id" : [2, 4],
    "return_status" : ["Yes", "No"]
})

returns.to_csv("../data/return.csv", index=False)

In [13]:
df = pd.DataFrame(returns)

In [ ]:
# Did Install MySQL Connector

# Now code and fetchc from the mySQL dB 


In [14]:
import mysql.connector


conn = mysql.connector.connect(
    host="mysql_host", 
    user="mysql_user", 
    password="mysql_pass",
    database="mysql_db",
    
)



# Query the database

cursor = conn.cursor()

cursor.execute("SELECT* FROM orders")

for row in cursor.fetchall():
    print(row)

cursor.close()

conn.close()





 
 
# query = 'SELECT * FROM orders;'
# # orders_df = pd.read_sql(query, conn)

# # conn.close()



# # print(orders_df.head())

(1, 1, 101, 2, '2025-09-01')
(2, 2, 102, 1, '2025-09-02')
(3, 3, 103, 4, '2025-09-07')
(4, 1, 104, 1, '2025-09-07')


In [16]:
# Exporting a MySQL Database to .sql File 


import os

# DB credentials
user = "mysql_user"
password = "mysql_pass"
database = "mysql_db"
output_file = "C:\data-blending-project\sql\creat_orders.sql"

# Run mysqldump
cmd = f'mysqldump -u {user} -p{password} {database} > "{output_file}"'
os.system(cmd)

print(f"Database exported successfully to {output_file}")


<>:10: SyntaxWarning: invalid escape sequence '\d'
<>:10: SyntaxWarning: invalid escape sequence '\d'
C:\Users\User\AppData\Local\Temp\ipykernel_21100\1622984321.py:10: SyntaxWarning: invalid escape sequence '\d'
  output_file = "C:\data-blending-project\sql\creat_orders.sql"


Database exported successfully to C:\data-blending-project\sql\creat_orders.sql


In [17]:
sales_pd = pd.read_csv("../data/sales.csv")

In [18]:
print(sales_pd)

   order_id  customer_id  product_id  amount
0         1            1         101     200
1         2            2         102     150
2         3            3         103     300
3         4            1         104     250


In [2]:
import requests, json

api_key = "api_key"

# List of cities you want to collect
cities = ["Kuala Lumpur", "Singapore", "Jakarta", "Bangkok", "Manila"]

weather_list = []

for city in cities:
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    data = response.json()
    
    if data["cod"] != 200:
        print(f" No data for {city}")
        continue

    weather_list.append({
        "city": data["name"],
        "weather": data["weather"][0]["main"],
        "temperature_C": round(data["main"]["temp"], 1)
    })

# Save to JSON file
with open("../data/api_data.json", "w") as f:
    json.dump(weather_list, f, indent=4)

print(" Weather data saved for:", [w["city"] for w in weather_list])


 Weather data saved for: ['Kuala Lumpur', 'Singapore', 'Jakarta', 'Bangkok', 'Manila']


In [3]:
import pandas as pd
import json

with open("../data/api_data.json", "r") as f:
    weather_data = json.load(f)

weather_df = pd.DataFrame(weather_data)
print(weather_df)




           city weather  temperature_C
0  Kuala Lumpur  Clouds           28.4
1     Singapore  Clouds           28.5
2       Jakarta  Clouds           28.9
3       Bangkok  Clouds           29.0
4        Manila  Clouds           26.7


In [4]:
# Step 4 Data Blending Pipeline



import pandas as pd
import json
from sqlalchemy import create_engine


#  CSVs
sales_df = pd.read_csv("../data/sales.csv")
customers_df = pd.read_csv("../data/customers.csv")
returns_df = pd.read_csv("../data/return.csv")

# MySQL with SQLAlchemy

engine = create_engine("mysql+mysqlconnector://{mysql_user}:{mysql_pass}@l{mysql_host}:3306/{mysql_db}")

orders_df = pd.read_sql("SELECT * FROM orders;", engine)


# API (Weather JSON) 

with open("../data/api_data.json", "r") as f:
    weather_data = json.load(f)

weather_df = pd.DataFrame(weather_data)

print(" Data Loaded")



 Data Loaded


In [5]:

import pandas as pd

#  Load the CSV file
df = pd.read_csv("../data/customers.csv")

# Update the city for the customer with id 

df.loc[df['customer_id'] == 1, 'region'] = "Singapore"
df.loc[df['customer_id'] == 2, 'region'] = "Jakarta"
df.loc[df['customer_id'] == 3, 'region'] = "Manila"





# 3. Save changes back to CSV
df.to_csv("../data/customers.csv", index=False)

print("City updated successfully in CSV!")




City updated successfully in CSV!


In [6]:
# Blend CSV + Mysql
import pandas as pd

sales_customers = pd.merge(sales_df, customers_df, on="customer_id", how="left")

sales_orders = pd.merge(sales_customers,orders_df, on=["order_id", "customer_id","product_id"], how="left")

sales_orders_returns = pd.merge(sales_orders, returns_df, on="order_id", how="left")

print(sales_orders_returns.head())



   order_id  customer_id  product_id  amount     name     region  quantity  \
0         1            1         101     200    Alice  Singapore         2   
1         2            2         102     150      Bob    Jakarta         1   
2         3            3         103     300  Charlie     Manila         4   
3         4            1         104     250    Alice  Singapore         1   

   order_date return_status  
0  2025-09-01           NaN  
1  2025-09-02           Yes  
2  2025-09-07           NaN  
3  2025-09-07            No  


In [7]:
# Merge with weather (assuming region = city)
final_df = pd.merge(
    sales_orders_returns,
    weather_df,
    left_on="region",
    right_on="city",
    how="left"
)

print(final_df.head())



   order_id  customer_id  product_id  amount     name     region  quantity  \
0         1            1         101     200    Alice  Singapore         2   
1         2            2         102     150      Bob    Jakarta         1   
2         3            3         103     300  Charlie     Manila         4   
3         4            1         104     250    Alice  Singapore         1   

   order_date return_status       city weather  temperature_C  
0  2025-09-01           NaN  Singapore  Clouds           28.5  
1  2025-09-02           Yes    Jakarta  Clouds           28.9  
2  2025-09-07           NaN     Manila  Clouds           26.7  
3  2025-09-07            No  Singapore  Clouds           28.5  


In [8]:
final_df.to_csv("../data/blended_final.csv", index=False)
print(" Final blended dataset saved as blended_final.csv")



 Final blended dataset saved as blended_final.csv


In [9]:

# Analaysis


# Total sales by city
print(final_df.groupby("city")["amount"].sum())

# Return status counts
print(final_df.groupby("city")["return_status"].value_counts())

# Average temperature per city
print(final_df.groupby("city")["temperature_C"].mean())


city
Jakarta      150
Manila       300
Singapore    450
Name: amount, dtype: int64
city       return_status
Jakarta    Yes              1
Singapore  No               1
Name: count, dtype: int64
city
Jakarta      28.9
Manila       26.7
Singapore    28.5
Name: temperature_C, dtype: float64
